# WAWE (Wasserstein SPN Weight Estimation) 调用测试

这个 Notebook 用来测试如何以编程方式调用 `ot_backprop_pnwo` 的核心功能。

**重要：** 在运行之前，请确保你已经将一个 XES 事件日志文件和一个 PNML 模型文件放入与此 Notebook 同级的 `data` 目录中，并更新下面代码单元格中的 `path_log` 和 `path_pn` 变量。

### 1. 导入依赖

In [3]:
from pathlib import Path
from ot_backprop_pnwo import run_wawe
from ot_backprop_pnwo.evaluation.evaluation_param import ConvergenceConfig
from ot_backprop_pnwo.optimization.emsc_loss_type import EMSCLossType
from ot_backprop_pnwo.optimization.model import Path2VariantLayerTypes, ResidualHandling

print("依赖导入成功！")

依赖导入成功！


In [4]:
# --- 请在此处修改你的文件名 --- #
log_file_name = "Depart.xes"       # <--- 修改这里
net_file_name = "ori.pnml"     # <--- 修改这里
# -------------------------------- #

path_log = Path(f'./data/{log_file_name}')
path_pn = Path(f'./data/{net_file_name}')
path_output = Path(f'./data/optimized_{net_file_name}')

# 配置优化参数: (min_iterations, max_iterations, convergence_threshold)
conv_config = ConvergenceConfig(nbr_iterations_min=50, nbr_iterations_max=5000, eps_convergence=0.0025)

print(f"日志文件路径: {path_log}")
print(f"模型文件路径: {path_pn}")
print(f"输出文件路径: {path_output}")

日志文件路径: data\Depart.xes
模型文件路径: data\ori.pnml
输出文件路径: data\optimized_ori.pnml


### 2. 设置路径和配置

In [5]:
# path_log = "./data/Depart.xes"
# path_pn = "./data/ori.pnml"
# path_output = "./data/optDepart.pnml"

In [6]:
run_wawe.main(path_log, path_pn, path_output, 
                emsc_loss_type=EMSCLossType.PEMSC, 
                max_nbr_paths=600, max_nbr_variants=600, 
                layer_type=Path2VariantLayerTypes.EXP_LOG_ABS, 
                residual_handling=ResidualHandling.ADD_RESIDUAL_ELEMENT, 
                conv_config=conv_config, 
                warm_start=False, 
                phase_two_enabled=False)
print("--- \n优化完成！--- ")
print(f"优化后的模型已保存到: {path_output}")

c:\Users\96513\anaconda3\envs\wasser\Lib\site-packages\pm4py\utils.py:992: UserWarning: Install the optional requirement `rustxes` to import/export files faster.
  warnings.warn("Install the optional requirement `rustxes` to import/export files faster.")
c:\Users\96513\anaconda3\envs\wasser\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 29297/29297 [00:00<00:00, 55159.32it/s]


--- 
优化完成！--- 
优化后的模型已保存到: data\optimized_ori.pnml


### 3. 运行优化

In [ ]:
print("开始运行优化过程...")

# 检查输入文件是否存在
if not path_log.exists() or not path_pn.exists():
    print("错误：找不到日志文件或模型文件。请确保文件存在于 'data' 目录中，并且文件名已在上面的单元格中正确设置。")
else:
    run_wawe.main(path_log, path_pn, path_output, 
                  emsc_loss_type=EMSCLossType.PEMSC, 
                  max_nbr_paths=600, max_nbr_variants=600, 
                  layer_type=Path2VariantLayerTypes.EXP_LOG_ABS, 
                  residual_handling=ResidualHandling.ADD_RESIDUAL_ELEMENT, 
                  conv_config=conv_config, 
                  warm_start=False, 
                  phase_two_enabled=False)
    print("--- \n优化完成！--- ")
    print(f"优化后的模型已保存到: {path_output}")

开始运行优化过程...


parsing log, completed traces :: 100%|██████████| 29297/29297 [00:00<00:00, 43326.20it/s]


--- 
优化完成！--- 
优化后的模型已保存到: data\optimized_ori.pnml
